In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
# Define the MLP model class
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(30, 64),  # input layer (30 features) -> hidden layer (64 neurons)
            nn.ReLU(),          # activation function
            nn.Linear(64, 1),   # hidden layer (64 neurons) -> output layer (1 neuron)
            nn.Sigmoid()        # sigmoid activation for binary classification
        )
    
    def forward(self, x):
        return self.model(x)

In [10]:
# Load Breast Cancer dataset from sklearn
data = load_breast_cancer()
X = data.data  # Features
y = data.target  # Labels (0 = benign, 1 = malignant)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Standardize the data (normalize)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [13]:
# Instantiate the model
mlp = MLP()

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(mlp.parameters(), lr=0.001)


In [14]:
# Training loop
epochs = 500
for epoch in range(epochs):
    # Forward pass
    outputs = mlp(X_train_tensor)
    
    # Compute the loss
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass
    optimizer.zero_grad()   # Zero the gradients before backpropagation
    loss.backward()         # Backpropagation to calculate gradients
    
    # Update weights
    optimizer.step()        # Update weights using optimizer
    
    # Print loss every 50 iterations
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [50/500], Loss: 0.2125
Epoch [100/500], Loss: 0.1114
Epoch [150/500], Loss: 0.0793
Epoch [200/500], Loss: 0.0635
Epoch [250/500], Loss: 0.0532
Epoch [300/500], Loss: 0.0458
Epoch [350/500], Loss: 0.0397
Epoch [400/500], Loss: 0.0345
Epoch [450/500], Loss: 0.0299
Epoch [500/500], Loss: 0.0259


In [15]:
# Evaluate the model on the test set
with torch.no_grad():  # Disable gradient tracking during testing
    y_pred = mlp(X_test_tensor)  # Get predictions
    y_pred_class = (y_pred > 0.5).float()  # Convert to binary (0 or 1)

    accuracy = (y_pred_class == y_test_tensor).float().mean()  # Compute accuracy
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

Test Accuracy: 97.37%
